# $Crank–Nicolson$
In numerical analysis, the Crank–Nicolson method is a finite difference method used for numerically solving the heat equation and similar partial differential equations.

Black-Scholes Partial Differential Equation (PDE): $rf = \frac{df}{dt} + rS\frac{df}{dS} + \frac{1}{2}\sigma^2 S^2 \frac{d^2 f}{dt^2}$

To solve PDE we introduce discrete-time grid of size $M$ by $N$, it will reflect prices over the time, when we move backwards in it, using finite differences scheme. Then $S$, and $t$ will carry on the values: $S = 0, 2dS, 3dS,..., (M−1)dS, S_{max}$ and $t = 0, dt, 2dt, 3dt,..., (N−1)dt, T$ respectfully.



$\frac{1}{2}rf_{i, j-1} + \frac{1}{2}rf_{i, j} = \frac{ f_{i, j} - f_{i, j-1} }{dt} \frac{1}{2}ridS \left( \frac{ f_{i+1, j-1} - f_{i-1, j-1} }{2dS} \right) + \frac{1}{2}ridS \left( \frac{ f_{i+1, j} - f_{i-1, j} }{2dS} \right) + \frac{1}{4}\sigma^2 i^2 dS^2 \left( \frac{ f_{i+1, j-1} - 2f_{i, j-1} + f_{i-1, j-1}}{dS^2}  \right) + \frac{1}{4}\sigma^2 i^2 dS^2 \left( \frac{ f_{i+1, j} - 2f_{i, j} + f_{i-1, j}}{dS^2}  \right)$

$
\epsilon_i = \frac{dt}{4}  \left( \sigma^2 i^2 - ri \right)\\

\zeta_i = \frac{dt}{2}  \left( \sigma^2 i^2 + ri \right)\\

\xi_i = \frac{dt}{4}  \left( \sigma^2 i^2 + ri \right) \\

$



$
-\epsilon_i f_{i-1, j-1} + (1 - \zeta_i)f_{i, j-1} - \xi_i f_{i+1, j-1} = \epsilon_i f_{i-1, j} + (1 - \zeta_i)f_{i, j-1} - \xi_i f_{i+1, j}
$

It follows by constructing the following system of linear equations:
$
M_1 f_{j-1} = M_2 f_{j}
$


$
M_1 = \left[
\begin{array}{rrrrrr}
1-\zeta_1& -\xi_1& 0& 0& 0& 0\\
-\epsilon_2& 1-\zeta_2& -\xi_2& 0& 0& 0\\
0& -\epsilon_3& 1-\zeta_3& -\xi_3& 0& 0&\\
0& 0& ...& ...& ...& 0\\
0& 0& 0& -\epsilon_{M-2}& 1-\zeta_{M-2}& -\xi_{M-2}&\\
0& 0& 0& 0& -\epsilon_{M-1}& 1-\zeta_{M-1}&
\end{array}

\right]
$

$
M_2 = \left[
\begin{array}{rrrrrr}
1+\zeta_1& \xi_1& 0& 0& 0& 0\\
\epsilon_2& 1+\zeta_2& \xi_2& 0& 0& 0\\
0& \epsilon_3& 1+\zeta_3& \xi_3& 0& 0&\\
0& 0& ...& ...& ...& 0\\
0& 0& 0& \epsilon_{M-2}& 1+\zeta_{M-2}& \xi_{M-2}&\\
0& 0& 0& 0& \epsilon_{M-1}& 1+\zeta_{M-1}&
\end{array}

\right]
$

$
f_i = \left[
\begin{array}{r}
f_{1, j}\\
f_{2, j}\\
.\\
.\\
.\\
f_{M - 1, j}\\
\end{array}

\right]
$

In [1]:
import OptionLib as opt
import numpy as np
import scipy.linalg as linalg

In [2]:
class CrankNicolson():
    def __init__(self, S0, K, r, T, sigma, Smax, M, N, type='call'):
        self.epsilon = None
        self.zeta = None
        self.xi = None
        self.S0 = S0
        self.K = K
        self.r = r
        self.T = T
        self.sigma = sigma
        self.Smax = Smax
        self.M = int(M)
        self.N = int(N)
        self.type = type
        self.dS = Smax / float(self.M)
        self.dt = T / float(self.N)
        self.i_values = np.arange(self.M)
        self.j_values = np.arange(self.N)
        self.grid = np.zeros(shape=(self.M + 1, self.N + 1))
        self.bnds = np.linspace(0, Smax, self.M + 1)  # we need to set some boundaries

    def set_greeks(self):
        self.epsilon = 0.25 * self.dt * ( (self.sigma ** 2) * (self.i_values ** 2) - self.r * self.i_values)
        self.zeta = -self.dt * 0.5 * ((self.sigma ** 2) * (self.i_values ** 2) + self.r)
        self.xi = 0.25 * self.dt * ((self.sigma ** 2) * (self.i_values ** 2) + self.r * self.i_values)

        self.M1 = -np.diag(self.epsilon[2:self.M], -1) + np.diag(1 - self.zeta[1:self.M]) - np.diag(self.xi[1:self.M - 1], 1)

        self.M2 = np.diag(self.epsilon[2:self.M], -1) + np.diag(1 + self.zeta[1:self.M]) + np.diag(self.xi[1:self.M - 1], 1)

    def traverse(self):
        # traverse babe, traverse https://www.youtube.com/watch?v=qbMFZ6SLX48
        P, L, U = linalg.lu(self.M1)
        for j in reversed(range(self.N)):
            x1 = linalg.solve(L, np.dot(self.M2, self.grid[1: self.M, j + 1]))
            x2 = linalg.solve(U, x1)
            self.grid[1: self.M, j] = x2

    def set_boundaries(self):

        if self.type == 'call':
            self.grid[:, -1] = np.maximum(self.bnds - self.K, 0)
            self.grid[-1, :-1] = (self.Smax - self.K) * np.exp(-self.r * self.dt * (self.N - self.j_values))

        elif self.type == 'put':
            self.grid[:, -1] = np.maximum(self.K - self.bnds, 0)
            self.grid[0, :-1] = (self.K - self.Smax) * np.exp(-self.r * self.dt * (self.N - self.j_values))

    def interpolation(self): return np.interp(self.S0, self.bnds, self.grid[:, 0])

    def price(self):
        self.set_boundaries()
        self.set_greeks()
        self.traverse()
        return self.interpolation()


In [3]:
# S0, K,   r,    T,   sigma, Smax, M, N, type
option = CrankNicolson(50, 50, 0.1, 5. / 12., 0.4, 100, 100, 100, 'put')
print(option.price())

# BSM(r, spot, strike, time, sigma, type='call'):
print(opt.BSM(r=.1, spot=50, strike=50, time=5/12, sigma=.4, type='put'))

4.072254507998117
4.075980984787783


Such result exhaustively explains Duffy in "Finite difference methods" by criticizing CN method for the following reasons:
$\bullet$ The Crank-Nicolson method is second-order accurate on uniform meshes only. \
$\bullet$ It gives terrible results near the stike price for approximations to the first and second derivatives in the space direction.
$\bullet$ In pricing applications, this translates to the statement that the Crank-Nicolson method gives bad approximations to the delta and gamma of the option price.